References:

- https://www-s.acm.illinois.edu/sigart/docs/QLearning.pdf
- http://webdocs.cs.ualberta.ca/~sutton/book/ebook/node65.html

In [76]:
print "*|*|*\n-----\n*|*|*\n-----\n*|*|*\n"

*|*|*
-----
*|*|*
-----
*|*|*



In [77]:
import numpy as np

def get_idx(state_type, game_state):
    assert state_type in ["O","X","*"]
    return np.where(state_type == np.array(list(game_state)))[0].tolist()

def check_game_state(game_state):
    assert len(game_state) <= 9, "game_state more than 9 characters"
    assert len(get_idx("O",game_state)) - len(get_idx("X",game_state)) in [0,1], "invalid game state"
    
    if len(game_state) < 9:
        game_state = game_state + "*"*(9-len(game_state))
        
    return game_state
    

class State(object):
    def __init__(self, game_state="*"*9):
        self._state = game_state
        self._state_len = len(self._state)
    
    def __repr__(self):
        return self._state
    
    @property
    def _np_state(self):
        return np.array(list(self._state))
    
    def projection(self, idx):
        return State("".join(self._np_state[idx]))
    
    @property
    def total_steps(self):
        return self._state_len - len(get_idx("*", self._state))
    
    def get_idx(self,state_type):
        return get_idx(state_type, self._state)
    
    def count_state_type(self,state_type):
        return len(get_idx(state_type, self._state))
    
    
    @property
    def X_index(self):
        return get_idx("X",self._state)
    
    @property
    def O_index(self):
        return get_idx("O",self._state)
    
    @property
    def free_index(self):
        return get_idx("*",self._state)
    

DEFAULT_CHECK_INDEX = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]

class BoardState(State):
    def __init__(self, game_state="*"*9):
        super(type(self),self).__init__(check_game_state(game_state))
        self.need_to_check_index = []
        self.need_to_check_index.extend(DEFAULT_CHECK_INDEX)
        self.prune_not_interesting_check_index
        
    def __repr__(self):
        return "State:\n{0}|{1}|{2}\n-----\n{3}|{4}|{5}\n-----\n{6}|{7}|{8}\n".format(*self._state)
    
    @property
    def player(self):
        return "O" if self.total_steps % 2 == 0 else "X"
    
    @property
    def opposite_player(self):
        return "X" if self.total_steps % 2 == 0 else "O"
    
    
    def _check_state(self, state_type):
        return np.array(map(lambda cgs_idx:self.projection(cgs_idx).count_state_type(state_type),self.need_to_check_index))
    
    @property
    def state_summary(self):
        O_filled_idx = []
        X_filled_idx = []
    
        O_checked_idx = []
        X_checked_idx = []

        not_interesting_idx = []
    
    
        filled_idx = np.where(self._check_state("*") == 0 )[0]
    
    
        for idx in filled_idx:
            #print "[filled_idx] idx = ",idx
            nOs = len(self.projection(self.need_to_check_index[idx]).O_index)
            if nOs == 0:
                X_filled_idx.append(idx)
            elif nOs == 3:
                O_filled_idx.append(idx)
            else:
                not_interesting_idx.append(idx)
    
        semifilled_idx = np.where(self._check_state("*") == 1 )[0]
    
        for idx in semifilled_idx:
            #print "[semifilled_idx] idx = ",idx
            nOs = len(self.projection(self.need_to_check_index[idx]).O_index)
            if nOs == 0:
                X_checked_idx.append(idx)
            elif nOs == 2:
                O_checked_idx.append(idx)
            else:
                not_interesting_idx.append(idx)
    
        #not_interesting_idx = list(set(not_interesting_idx))
        not_interesting_idx.sort()

        summary = {"filled":{"O":O_filled_idx,"X":X_filled_idx},
                   "checked":{"O":O_checked_idx,"X":X_checked_idx},
                   "not_interesting":not_interesting_idx}
        
        return summary
        
    @property
    def is_terminated(self):
        terminated = len(self.state_summary["filled"]["O"] + self.state_summary["filled"]["X"]) > 0 
        terminated = terminated  or len(self.free_index) == 0
        return terminated
    
    @property
    def winner(self):
        assert self.is_terminated == True, "game is not terminate!"
        if len(self.state_summary["filled"][self.opposite_player]) > 0: 
            return self.opposite_player
        elif len(self.state_summary["filled"][self.player]) > 0:
            return self.player
        else:
            return None
        
    @property
    def prune_not_interesting_check_index(self):
        summary = self.state_summary
        remove_idx = [self.need_to_check_index[i] for i in summary["not_interesting"]]
        for idx in remove_idx:
            self.need_to_check_index.remove(idx)
    
    
    def game_reward(self, player):
        assert player in ["O","X"],'player must in ["O","X"]'
        if self.is_terminated:
            if self.winner:
                if self.winner == player:
                    return 100
                else:
                    return -100

        return 0
    
    @property
    def actions(self):
        if self.is_terminated:
            return []
        else:
            return self.free_index
    
    
    def take_action(self, action):
        assert action in self.actions, "action is not in self.actions"
        new_state = list(self._state)
        new_state[action] = self.player
        new_state = "".join(new_state)
        return type(self)(new_state)
        

In [78]:
s1 = BoardState("O*X*O*OX*")
s2 = BoardState("XOX*O*O**")
s3 = BoardState("O*X*O*OXX")
s = BoardState("O*XOO*OXX")

In [79]:
s

State:
O|*|X
-----
O|O|*
-----
O|X|X

In [81]:
map(lambda a:s.take_action(a), s.actions)

[]

In [82]:
s2.actions

[3, 5, 7, 8]

In [83]:
s2.take_action(8)

State:
X|O|X
-----
*|O|*
-----
O|*|X

In [84]:
map(lambda a:s3.take_action(a).game_reward("X"), s3.actions)

[0, -100, 0]

In [85]:
s3

State:
O|*|X
-----
*|O|*
-----
O|X|X

In [86]:
s1.O_index

[0, 4, 6]

In [87]:
s1.X_index

[2, 7]

In [88]:
s1.free_index

[1, 3, 5, 8]

In [89]:
s1.player

'X'

In [90]:
s1._state

'O*X*O*OX*'

In [44]:
s1

State:
O|*|X
-----
*|O|*
-----
O|X|*

In [34]:
s.game_reward("X")

-100

In [35]:
s.winner

'O'

In [38]:
s1.free_index

[1, 3, 5, 8]

In [4]:
ss = BoardState()

In [5]:
ss.state_summary

{'checked': {'O': [], 'X': []},
 'filled': {'O': [], 'X': []},
 'not_interesting': []}

In [6]:
s.is_terminated

True

In [7]:
s.need_to_check_index[s.state_summary["checked"][s.player][0]]


[2, 5, 8]

In [8]:
s.need_to_check_index

[[3, 4, 5], [0, 3, 6], [2, 5, 8]]

In [9]:
s

State:
O|*|X
-----
O|O|*
-----
O|X|X

In [10]:
s.prune_not_interesting_check_index

In [11]:
s1.state_summary

{'checked': {'O': [1, 3], 'X': []},
 'filled': {'O': [], 'X': []},
 'not_interesting': []}

In [12]:
s.winner

'O'

In [13]:
s1.free_index

[1, 3, 5, 8]

In [14]:
s1.state_summary

{'checked': {'O': [1, 3], 'X': []},
 'filled': {'O': [], 'X': []},
 'not_interesting': []}

In [15]:
s1.state_summary["checked"]["O"]

[1, 3]

In [16]:
s1.is_terminated

False

In [17]:
s2

State:
X|O|X
-----
*|O|*
-----
O|*|*

In [18]:
s2.state_summary

{'checked': {'O': [2], 'X': []},
 'filled': {'O': [], 'X': []},
 'not_interesting': []}

In [20]:
s2.is_terminated

False

In [19]:
s1.player

'X'

In [12]:
s2.need_to_check_index

[[3, 4, 5], [6, 7, 8], [1, 4, 7], [2, 5, 8]]

In [398]:
s.state_summary

{'checked': {'O': [1], 'X': [5]},
 'filled': {'O': [3], 'X': []},
 'not_interesting': [0, 2, 4, 6, 7]}

In [399]:
filled_idx

array([2, 6, 7])

In [315]:
semifilled_idx = np.where(s.check_state("*") == 1 )[0]

AttributeError: 'BoardState' object has no attribute 'check_state'

In [316]:
semifilled_idx

array([0, 3, 4, 5])

In [272]:
O_filled_idx

[]

In [273]:
X_filled_idx

[]

In [275]:
semifilled_idx

array([0, 3, 4, 5])

In [279]:
not_interesting_idx.sort()

In [280]:
not_interesting_idx

[0, 2, 4, 6, 7]

In [281]:
O_semifilled_idx

[3]

In [282]:
X_semifilled_idx

[5]

In [247]:
not_interesting_idx.append(4)

In [249]:
not_interesting_idx.sort()

In [250]:
not_interesting_idx

[0, 4, 7]

In [231]:
not_interesting_idx.pop()

IndexError: pop from empty list

In [236]:
s.need_to_check_index.pop(0)

[0, 1, 2]

In [237]:
s.need_to_check_index

[[3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8]]

In [234]:
not_interesting_idx(7)

TypeError: 'list' object is not callable

In [217]:
check_score = s.check_state("*")

In [198]:
filled_idx = np.where(check_score == 0)[0]

In [196]:
p = np.array(s.need_to_check_index)[np.where(check_score == 0)[0]]

In [189]:
len(s.projection(p[0]).get_idx("O")) in [0,3]

False

In [181]:
np.array(s.need_to_check_index)[np.where(check_score == 2)[0]]

array([[3, 4, 5],
       [2, 5, 8]])

In [168]:
s

State:
O|*|X
-----
*|O|*
-----
O|X|*

In [147]:
s.need_to_check_index

[[0, 1, 2],
 [3, 4, 5],
 [6, 7, 8],
 [0, 3, 6],
 [1, 4, 7],
 [2, 5, 8],
 [0, 4, 8],
 [2, 4, 6]]

In [148]:
s.next_player

'X'

In [149]:
s.free_index

[1, 3, 5, 8]

In [173]:
s.need_to_check_index

[[0, 1, 2],
 [3, 4, 5],
 [6, 7, 8],
 [0, 3, 6],
 [1, 4, 7],
 [2, 5, 8],
 [0, 4, 8],
 [2, 4, 6]]

In [174]:
s

State:
O|*|X
-----
*|O|*
-----
O|X|*

In [175]:
np.array(map(lambda cgs_idx:s.projection(cgs_idx).get_idx("O"),s.need_to_check_index))

array([[0], [1], [0], [0, 2], [1], [], [0, 1], [1, 2]], dtype=object)

In [177]:
np.array(map(lambda cgs_idx:s.projection(cgs_idx).get_idx("*"),s.need_to_check_index))

array([[1], [0, 2], [2], [1], [0], [1, 2], [2], []], dtype=object)

In [131]:
check_df = pd.DataFrame(s.need_to_check_index)

In [139]:
check_df.applymap(lambda cgs_idx:s.projection(cgs_idx))

,0,1,2
0,O,*,X
1,*,O,*
2,O,X,*
3,O,*,O
4,*,O,X
5,X,*,*
6,O,O,*
7,X,O,O


In [73]:
s.projection([1,4,7]).O_index

[]

In [61]:
s.free_index

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [62]:
s._np_state[[1,4,7]]

array(['*', '*', '*'], 
      dtype='|S1')

In [3]:
import numpy as np

In [4]:
s = np.array("*"*9)

In [5]:
s.shape = (3,3)

ValueError: total size of new array must be unchanged

In [58]:
s = State("OXO")

In [59]:
s

State:
O|X|O
-----
*|*|*
-----
*|*|*

In [60]:
s.next_player

'X'

In [15]:
import numpy as np

[0, 2, 4, 6]

In [6]:
import re

In [12]:
pat = re.compile("X")

['X', 'X', 'X', 'X']

In [7]:
re.findall("X","XOXOXOX")

['X', 'X', 'X', 'X']

In [8]:
re.findall?

In [9]:
re.finditer?